In [2]:
!pip install datasets


In [3]:
!pip install datasets transformers


In [4]:
from datasets import load_dataset

dataset = load_dataset('Kaludi/Customer-Support-Responses')

print(dataset['train'].features)
print(dataset['train'][0])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'query': Value(dtype='string', id=None), 'response': Value(dtype='string', id=None)}
{'query': "My order hasn't arrived yet.", 'response': 'We apologize for the inconvenience. Can you please provide your order number so we can investigate?'}


In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

def preprocess(data):
    inputs = [query for query in data['query']]
    outputs = [response for response in data['response']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding=True)
    model_outputs = tokenizer(outputs, max_length=512, truncation=True, padding=True)
    return model_inputs, model_outputs

train_inputs, train_outputs = preprocess(dataset['train'])


In [6]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load model and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Set the padding token
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    inputs = tokenizer(examples['query'], padding="max_length", truncation=True)
    outputs = tokenizer(examples['response'], padding="max_length", truncation=True)
    inputs['labels'] = outputs['input_ids']
    return inputs

# Tokenize the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)


In [8]:
!pip install transformers[torch] accelerate -U


In [3]:
# Install necessary libraries
!pip install transformers[torch] accelerate -U
!pip install datasets

# Import necessary modules
from datasets import load_dataset, DatasetDict
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments

# Load dataset
dataset = load_dataset('Kaludi/Customer-Support-Responses')

# Split the dataset manually
split_dataset = dataset['train'].train_test_split(test_size=0.1)
dataset = DatasetDict({
    'train': split_dataset['train'],
    'test': split_dataset['test']
})

# Load model and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Set the padding token
tokenizer.pad_token = tokenizer.eos_token

# Define the tokenize function
def tokenize_function(examples):
    inputs = tokenizer(examples['query'], padding="max_length", truncation=True)
    outputs = tokenizer(examples['response'], padding="max_length", truncation=True)
    inputs['labels'] = outputs['input_ids']
    return inputs

# Tokenize the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test']
)

# Start training
trainer.train()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 21.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylin

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/74 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/66 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Step,Training Loss


TrainOutput(global_step=51, training_loss=0.6570293202119715, metrics={'train_runtime': 77.0527, 'train_samples_per_second': 2.57, 'train_steps_per_second': 0.662, 'total_flos': 103471644672000.0, 'train_loss': 0.6570293202119715, 'epoch': 3.0})

In [6]:
import torch

def generate_response(input_text):
    # Ensure the model is on the same device as the input
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    tokenizer.pad_token_id = tokenizer.eos_token_id

    # Prepare inputs and move to the appropriate device
    inputs = tokenizer.encode(input_text, return_tensors='pt').to(device)

    # Generate outputs
    outputs = model.generate(inputs, max_length=50, num_return_sequences=1)

    # Decode and return the generated text
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test the model
print(generate_response("How can I reset my password?"))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


How can I reset my password? please email me.


In [9]:
# Install the necessary libraries
!pip install rouge_score nltk -U

# Import necessary modules
from datasets import load_dataset, DatasetDict
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import load_metric
import torch

# Load dataset
dataset = load_dataset('Kaludi/Customer-Support-Responses')

# Split the dataset manually
split_dataset = dataset['train'].train_test_split(test_size=0.1)
dataset = DatasetDict({
    'train': split_dataset['train'],
    'test': split_dataset['test']
})

# Load model and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Set the padding token
tokenizer.pad_token = tokenizer.eos_token

# Define the tokenize function
def tokenize_function(examples):
    inputs = tokenizer(examples['query'], padding="max_length", truncation=True)
    outputs = tokenizer(examples['response'], padding="max_length", truncation=True)
    inputs['labels'] = outputs['input_ids']
    return inputs

# Tokenize the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test']
)

# Start training
trainer.train()

# Define the response generation function
def generate_response(input_text):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    tokenizer.pad_token_id = tokenizer.eos_token_id
    inputs = tokenizer.encode(input_text, return_tensors='pt').to(device)
    outputs = model.generate(inputs, max_length=50, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Load evaluation metrics
from datasets import load_metric
rouge = load_metric('rouge')
bleu = load_metric('bleu')

# Define the evaluation function
def evaluate_model(predictions, references):
    predictions = [{'prediction_text': pred} for pred in predictions]
    references = [{'reference_text': ref} for ref in references]
    rouge_score = rouge.compute(predictions=predictions, references=references)
    bleu_score = bleu.compute(predictions=[pred['prediction_text'].split() for pred in predictions],
                              references=[[ref['reference_text'].split()] for ref in references])
    return rouge_score, bleu_score

# Generate predictions and references
preds = [generate_response(query) for query in dataset['test']['query']]
refs = [response for response in dataset['test']['response']]

# Evaluate the model
rouge_score, bleu_score = evaluate_model(preds, refs)
print("ROUGE Score:", rouge_score)
print("BLEU Score:", bleu_score)


Step,Training Loss


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

ROUGE Score: {'rouge1': AggregateScore(low=Score(precision=0.36485697751322754, recall=0.17344166318283163, fmeasure=0.2327540907557355), mid=Score(precision=0.46623677248677253, recall=0.24964502624489038, fmeasure=0.3201923076923077), high=Score(precision=0.5648164682539683, recall=0.33152250557413604, fmeasure=0.3978207236842105)), 'rouge2': AggregateScore(low=Score(precision=0.027777777777777776, recall=0.010416666666666666, fmeasure=0.015151515151515152), mid=Score(precision=0.09908804645646752, recall=0.05595466155810983, fmeasure=0.06973684210526315), high=Score(precision=0.17125736105999265, recall=0.10938784220121119, fmeasure=0.12616427432216903)), 'rougeL': AggregateScore(low=Score(precision=0.2910358796296296, recall=0.13894565798030473, fmeasure=0.18661310728744934), mid=Score(precision=0.36813822751322745, recall=0.19418096697988002, fmeasure=0.24985902255639098), high=Score(precision=0.46234375, recall=0.2870813707729469, fmeasure=0.3332239553209948)), 'rougeLsum': Aggre

In [10]:
model.save_pretrained('./customer_support_model')
tokenizer.save_pretrained('./customer_support_tokenizer')


('./customer_support_tokenizer/tokenizer_config.json',
 './customer_support_tokenizer/special_tokens_map.json',
 './customer_support_tokenizer/vocab.json',
 './customer_support_tokenizer/merges.txt',
 './customer_support_tokenizer/added_tokens.json')

In [11]:
# In Jupyter notebook
input_query = "How do I change my subscription plan?"
response = generate_response(input_query)
print(f"Query: {input_query}\nResponse: {response}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Query: How do I change my subscription plan?
Response: How do I change my subscription plan?
